In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import Prepare_Data
import pyarrow 
from Prepare_Data import load_and_filter_market_returns
import data_run_files
import os
#autoreload download #pip install ipython
%load_ext autoreload
%autoreload 2

#file_path_id_test="./data_test/top_5_percent_ids.csv"
# Sti til usa_dsf.parquet
file_path_usa_dsf = "./Data/usa_dsf.parquet"
file_path_usa = "./Data/usa.parquet"
file_path_market_returns = "./Data/market_returns.csv"
file_path_crsp_a_stock_with_id = "./Data/crsp_a_stock_codes.csv"
file_path_cluster_labels = "./Data/Cluster Labels.csv"
file_path_factor_details = "./Data/Factor Details.xlsx"
file_path_tsla = "./Data/tsla101.csv"
file_path_world_ret = "./Data/world_ret_monthly.csv"
print(1)

Features list length: 154
1


In [2]:
#Input paths
rente_path = "Data/ff3_m.csv"

In [3]:
folder_name = "much_more_data"

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    print(f"Folder '{folder_name}' created.")
else:
    print(f"Folder '{folder_name}' already exists.")

Folder 'much_more_data' already exists.


In [4]:
# output_path
risk_free_test_path = os.path.join(folder_name, "risk_free_test.csv")
output_path_usa_dsf = os.path.join(folder_name, "usa_dsf_test.parquet")
output_path_usa = os.path.join(folder_name, "usa_test.parquet")
output_path_market_returns = os.path.join(folder_name, "market_returns_test.csv")
world_ret_thres_test_path = os.path.join(folder_name, "world_ret_test.csv")

In [5]:
#Dates
start_date = pd.to_datetime('1952-12-31')
end_date = pd.to_datetime('2022-12-31')
#Threshold 
thres = 0.50 #(vi beholder de øverste 10%, hvis den er lig 0.9)

# Top_thres_ids.csv (csv fil med unikke id'er)

In [6]:
df_usa = pd.read_parquet(file_path_usa, engine='pyarrow')
print("Fil indlæst med succes.")

Fil indlæst med succes.


In [7]:
# Filtrer kun USA og id <= 99999
df_usa = df_usa[(df_usa["excntry"] == "USA") & (df_usa["id"] <= 99999)]

In [8]:
df_usa.loc[:, 'eom'] = pd.to_datetime(df_usa['eom'], errors='coerce')

df_usa_filtered = df_usa[df_usa['eom'] >= start_date]

print(df_usa_filtered.head())
print(f"Antal observationer efter filtrering: {len(df_usa_filtered)}")

# Valider at filtreringen virker
print(df_usa_filtered['eom'].min())  # Skal vise en dato >= start_date

      eom_ret     id                  eom  valid_ret excntry         me   sic  \
4  1953-01-31  10006  1952-12-31 00:00:00       True     USA  24.218250  3740   
5  1953-02-28  10006  1953-01-31 00:00:00       True     USA  23.394500  3740   
6  1953-03-31  10006  1953-02-28 00:00:00       True     USA  26.030500  3740   
7  1953-04-30  10006  1953-03-31 00:00:00       True     USA  24.053500  3740   
8  1953-05-31  10006  1953-04-30 00:00:00       True     USA  25.124375  3740   

  size_grp  ret_exc_lead1m  niq_su  ...     ni_be    ocf_at  ocf_at_chg1  \
4    small       -0.035634     NaN  ...  0.100695  0.084642     0.038812   
5    small        0.111274     NaN  ...  0.100695  0.084642     0.038812   
6    small       -0.058854     NaN  ...  0.100695  0.084642     0.038812   
7    small        0.042885     NaN  ...  0.100695  0.084642     0.038812   
8    small        0.263852     NaN  ...  0.100695  0.084642     0.038812   

   mispricing_perf  mispricing_mgmt  qmj  qmj_prof  qmj_

In [9]:
# Beregn median market equity for hver virksomhed
company_median_equity = df_usa_filtered.groupby('id')['market_equity'].median()

# Find 95. percentilen for virksomheders median market equity
threshold = company_median_equity.quantile(thres)

# Udvælg top 5% af virksomhederne (dem med median market equity >= threshold)
top_thres_ids = company_median_equity[company_median_equity >= threshold].index

# Filtrer df_usa_filtered, så det kun indeholder disse virksomheder
df_thres_ids = df_usa_filtered[df_usa_filtered['id'].isin(top_thres_ids)]

# Udskriv information
print(f"Grænseværdi for top 15% virksomheder: {threshold}")
print(f"Antal unikke virksomheder i top 15%: {len(top_thres_ids)}")

Grænseværdi for top 15% virksomheder: 545.8998885925
Antal unikke virksomheder i top 15%: 6160


In [10]:
top_thres_ids

Index([10026, 10032, 10051, 10078, 10085, 10104, 10107, 10108, 10119, 10126,
       ...
       93401, 93402, 93418, 93420, 93422, 93423, 93427, 93428, 93429, 93436],
      dtype='int32', name='id', length=6160)

In [11]:
top_thres_ids.unique()

Index([10026, 10032, 10051, 10078, 10085, 10104, 10107, 10108, 10119, 10126,
       ...
       93401, 93402, 93418, 93420, 93422, 93423, 93427, 93428, 93429, 93436],
      dtype='int32', name='id', length=6160)

In [14]:
# Find første og sidste dato i dataset
first_date = df_thres_ids['eom'].min()
first_date2 = pd.to_datetime('2000-12-31')
print(first_date2)
last_date = df_thres_ids['eom'].max()
print(last_date)
# Filtrer virksomheder, der har data på både første og sidste dato
df_thres_ids = df_thres_ids.groupby('id').filter(
    lambda x: (first_date2 in x['eom'].values) and (last_date in x['eom'].values)
)
# Udskriv antal unikke virksomheder efter filtrering
print(f"Antal unikke virksomheder med data på både første og sidste dato: {df_thres_ids['id'].nunique()}")

2000-12-31 00:00:00
2023-11-30 00:00:00
Antal unikke virksomheder med data på både første og sidste dato: 740


In [36]:
df_ids = pd.DataFrame(df_thres_ids['id'].unique(), columns=['id'])
df_ids_path = os.path.join(folder_name, "top_5_percent_ids.csv")
# Gem til en CSV-fil uden index-kolonne
df_ids.to_csv(df_ids_path, index=False)

    
print("Fil gemt som 'top_5_percent_ids.csv'")

Fil gemt som 'top_5_percent_ids.csv'


# De andre filer (med antal id's fra thres og start-end date)

In [37]:
file_path_id_test = os.path.join(folder_name, "top_5_percent_ids.csv")
file_path_usa = "./Data/usa_rvol.parquet" #Vi skal have den med rvol som er med i

In [38]:
risk_free = data_run_files.process_risk_free_rate(rente_path, start_date, end_date, output_path = risk_free_test_path)
# Filtrér ID'er for usa_dsf og usa
data_run_files.filter_ids_from_dataset(file_path_usa_dsf, file_path_id_test, output_path_usa_dsf, start_date)
data_run_files.filter_ids_from_dataset(file_path_usa, file_path_id_test, output_path_usa, start_date)
data_run_files.filter_and_save_data(file_path_market_returns, start_date, end_date, output_path_market_returns)
h_list = [1]  # Horisonter
dataret = data_run_files.monthly_returns(risk_free, h_list, output_path_usa)
data_run_files.world_ret_monthly_test_filter(file_path_id_test, file_path_world_ret, start_date, end_date, output_file = world_ret_thres_test_path)

Filtrering udført. Antal rækker efter filtrering: 841
Fil gemt som much_more_data\risk_free_test.csv
Fil ./Data/usa_dsf.parquet indlæst med succes. Antal rækker før filtrering: 176877668
Filtrering af ID'er udført. Antal rækker efter filtrering: 8415605
Bruger 'date' i stedet for 'eom'.
Filtrering på startdato 1952-12-31 00:00:00 udført. Antal rækker efter filtrering: 7985109
Antal unikke virksomheder efter filtrering: 740
Fil gemt som much_more_data\usa_dsf_test.parquet
Fil ./Data/usa_rvol.parquet indlæst med succes. Antal rækker før filtrering: 1373911
Filtrering af ID'er udført. Antal rækker efter filtrering: 336020
Filtrering på startdato 1952-12-31 00:00:00 udført. Antal rækker efter filtrering: 335389
Antal unikke virksomheder efter filtrering: 740
Fil gemt som much_more_data\usa_test.parquet
Filen er indlæst og filtreret succesfuldt.
Fil gemt som much_more_data\market_returns_test.csv med 841 rækker.
All missing excludes 5.68% of the observations
Data gemt i: much_more_data\worl